### 7. Bildbearbeitung 1

#### 7.1. Bilddatei lesen und speichern
<center>

![alt text](cpp1_a7_abb_7_1_dreifach.png)<br>
Abbildung 7.1.: Grauwertbild in der Datei `cpp1_a7_abb_7_1_dreifach.pgm`
</center>
In der Datei `cpp1_a7_abb_7_1_dreifach.pgm` ist ein Grauwertbild (Abbildung 7.1) der Größe 512 × 256 Bildpunkten
gespeichert. Der Dateianfang ist in Liste 7.1 dargestellt (siehe auch Tabelle 7.1). Eine Bildzeile (512
Grauwerte) ist dabei in 32 aufeinanderfolgenden Dateizeilen gespeichert.

<center>

Liste 7.1: Beginn der pgm-Datei `cpp1_a7_abb_7_1_dreifach.pgm`
![alt text](cpp1_a7_liste_7_1_dreifach.png)
</center>

Schreiben Sie ein Python-Skript mit einer Funktion in welcher die Datei `cpp1_a7_abb_7_1_dreifach.pgm` eingelesen wird. Die Bildinformationen (Zeile 1-3) sollen zur Kontrolle ausgegeben werden. Das Bild wird in eine Doppel-Liste (Liste aus Zeilen mit Liste aus Pixeln (Spalten)) (bild-)zeilenweise eingelesen (siehe Liste 7.2). Beim Einlesen ist zu überprüfen, ob die Codierung des Dateityps (hier P2) richtig ist und ob das Bild in die angegebenen Bild-Dimensionen passt. Im Fehlerfall ist das Programm mit einer entsprechenden Meldung abzubrechen.<br>
Anschließend ist das Bild im selben Dateiformat in eine Datei `dreifach.out.pgm` zu speichern. Hierbei soll überprüft werden, ob das Schreiben in die Ausgabedatei erfolgreich war. Im Fehlerfall ist das Programm mit einer entsprechenden Meldung abzubrechen.<br>

Wenn auf dem Rechner ein Standardprogramm für die Anzeige von Bildern installiert und konfiguriert ist, kann dieser zur Anzeige des Bildes von Python aus gestartet werden:
```
import subprocess
...
subprocess.call("open " + imageFileName, shell=True)
```
Verwenden dieses, um charakteristische Bildbereiche zur Kontrolle zu vergrößern.

In [13]:
import subprocess    
dict()

def readImage(imageFile):
    magic=""
    width=0
    height=0
    maxgray=0;
    pixels=[]
    itemCount=0
    rowIdx=0
    colIdx=0
    with open(imageFile, 'r', encoding='ascii') as file:
            text = file.read()
    for item in text.split():
        if (itemCount==0):
            magic=item
        elif (itemCount==1):
            width=int(item)
        elif (itemCount==2):
            height=int(item)
        elif (itemCount==3):
            maxgray=int(item)
        else:
            # Zeilen-Liste ermitteln
            row = pixels[rowIdx] if len(pixels) > rowIdx else []
            
            # Zeilenumbruch
            if len(row) >= width:
                rowIdx+=1
                colIdx=0
                row=[]
            # Pixelwert einfügen
            pixel = int(item)
            if (pixel < 0 or pixel > maxgray):
                raise ValueError(f"Pixel[{rowIdx}][{colIdx}]Erwarte Pixel 0..{maxgray}, ist aber {pixel}")
            
            row.append(int(item))
            colIdx+=1
            
            # Zeile anhängen, wenn neu
            if (len(row) == 1):
                pixels.append(row)
        itemCount+=1
    return {'magic' : magic, 'width' : width, 'height' : height, 'maxgray' : maxgray, 'pixels' : pixels}
try:
    infile="cpp1_a7_abb_7_1_dreifach.pgm"
    outfile="dreifach.out.pgm"

    image = readImage(infile)

    with open(outfile, 'w', encoding='ascii') as file:
        file.write(f"{image.get('magic')}\n")
        file.write(f"{image.get('width')}{image.get('height'):4d}\n")
        file.write(f"{image.get('maxgray')}\n")
        for row in image.get('pixels'):
            for pixel in row:
                file.write(f"{pixel:4d} ")
            file.write("\n")
    subprocess.call("open " + outfile, shell=True)
except ValueError as e:
    print(str(e))
except Exception as e:
    print("Unerwarteter Fehler:")
    print(str(e))
    raise e

kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found


kf.i18n.kuit: "Unknown subcue ':whatsthis,' in UI marker in context {@info:whatsthis, %1 the action's text}."
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-aptus-mos"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-arq"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-bay"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-bmq"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cap"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cine"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-cs1"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-dc2"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-drf"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-dxo"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-epson-eip"
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-epson-erf"
org.kde.kdegraphics.gwenview.li